In [2]:
import pandas as pd

# Загрузка датасета
file_path = 'sentiment_texts.pickle'
data = pd.read_pickle(file_path)
data = data[['issuerid', 'SentimentScore', 'MessageText']]

# Показываем первые несколько строк датасета для оценки его структуры
data.head()

,issuerid,SentimentScore,MessageText
0,153,2,⚠️🇷🇺#SELG #дивиденд сд Селигдар: дивиденды 20...
1,230,4,Ozon продолжает развивать специализированные ф...
2,118,4,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...
3,220,5,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...
4,89,2,​​Windfall Tax — налог на сверхприбыль. Какие ...


In [3]:
import json
import pathlib
import nltk
from nltk import word_tokenize, SnowballStemmer
nltk.download('punkt')

num_companies = 276

file_path = '/content/dict.json'
with open(file_path, 'r', encoding='utf-8') as file:
    synon_dict = json.load(file)

stemmer = SnowballStemmer("russian")

markers = [f'%{i}%' for i in range(num_companies)]


def process_text(text, synon_dict):
    tokens = word_tokenize(text, language='russian')
    tokens = [token.lower() for token in tokens]

    stem_tokens = [stemmer.stem(token) for token in tokens if len(token) > 1]

    # Находим компании и заменяем на маркеры
    return replace_phrases_with_markers(stem_tokens, tokens, maximize_phrase_coverage(stem_tokens, synon_dict, 5), markers)


def maximize_phrase_coverage(tokens: list[str], dictionary, max_window_size=20) -> list[list[int]]:
    def binary_search(target: str) -> int:
        if target in dictionary:
            return len(target.split())  # Фраза найдена
        return 0  # Фраза не найдена

    # Фильтруем только непустые токены
    if len(tokens) == 0:
        return []
    token_ids, text_tokens = zip(*[[idx, token] for idx, token in enumerate(tokens) if token])
    token_count = len(text_tokens)

    # max_sums[token_idx] - максимальное значение суммы значений для префикса [0...token_idx]
    max_sums = [0] * token_count
    window_sizes = [-1] * token_count

    # Инициализация для первого токена
    max_sums[0] = binary_search(text_tokens[0])

    for token_idx in range(1, token_count):
        current_window = ''
        for sz in range(1, max_window_size + 1):
            prev_token_idx = token_idx - sz
            if prev_token_idx < -1:
                break

            if current_window == '':
                current_window = text_tokens[prev_token_idx + 1]
            else:
                current_window = text_tokens[prev_token_idx + 1] + ' ' + current_window

            coverage_value = binary_search(current_window)

            if current_window == 'moex' and text_tokens[prev_token_idx] == '(' and text_tokens[prev_token_idx + 2] == ':':
                continue

            if coverage_value != 0 and max_sums[token_idx] < max_sums[prev_token_idx] + coverage_value:
                max_sums[token_idx] = max_sums[prev_token_idx] + coverage_value
                window_sizes[token_idx] = sz

    # Восстановление размеров окон токенов
    bounds = []
    current_index = token_count - 1
    while current_index != -1:
        window_size = window_sizes[current_index]
        bounds.append(window_size)
        window_size = 1 if window_size == -1 else window_size
        current_index -= window_size
    bounds.reverse()

    # Восстановление групп токенов
    token_groups = []
    current_index = 0
    for window_size in bounds:
        if window_size == -1:
            current_index += 1
            continue
        group = token_ids[current_index:current_index + window_size]
        token_groups.append(group)
        current_index += window_size

    return token_groups


def replace_phrases_with_markers(stem_tokens, tokens, groups, markers):
    """
    Функция находит и заменяет группы токенов, соответствующие записям в словаре,
    на маркеры, а также сохраняет позиции маркеров в тексте.

    Параметры:
        tokens (list[str]): Исходный список слов.
        groups (list[list[int]]): Список групп индексов, соответствующих найденным фразам.
        markers (dict): Словарь маркеров, где ключ - это индекс фразы в dictionary, а значение - маркер.

    Возвращает:
        tuple: Модифицированный список слов и словарь с позициями маркеров.
    """
    new_tokens = []
    marker_positions = {}
    last_index = 0  # Следим за последним индексом, который был добавлен в new_tokens

    for group in groups:
        if not group:
            continue
        # Проверяем, соответствует ли группа записи в словаре
        phrase = " ".join(stem_tokens[idx] for idx in group)
        marker_index = synon_dict[phrase]  # Получаем маркер для найденной фразы, если он есть

        if marker_index and marker_index <= 274:
            # Добавляем все токены до начала текущей группы
            new_tokens.extend(stem_tokens[last_index:group[0]])
            # Добавляем маркер в новый список
            # new_tokens.append(markers[marker_index])
            # Записываем позицию маркера
            if marker_index not in marker_positions:
                marker_positions[marker_index] = []
            marker_positions[marker_index].append(len(new_tokens) - 1)

            # Обновляем последний обработанный индекс
            last_index = group[-1] + 1

    # Добавляем все оставшиеся токены после последней группы
    new_tokens.extend(stem_tokens[last_index:])

    return new_tokens, marker_positions


def extract_company_context(tokens, mentioned_companies, window_size=3):
    # Словарь для хранения контекстов для каждой компании
    company_contexts = {}

    # Перебираем все упоминания компаний
    for company_id, positions in mentioned_companies.items():
        # Список контекстов для текущей компании
        contexts = []

        for pos in positions:
            # Определяем начало и конец контекстного окна
            start = max(0, pos - window_size)
            end = min(len(tokens), pos + window_size + 1)

            # Собираем слова в окне вокруг вхождения компании
            context_tokens = tokens[start:end]

            # Добавляем полученный контекст в список контекстов компании
            for tok in context_tokens:
                contexts.append(tok)

        # Записываем список контекстов для текущей компании в общий словарь
        company_contexts[company_id] = contexts

    return company_contexts



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
tokens = []
labels = []
for index, row in data.iterrows():
    words, mentioned_companies = process_text(row.MessageText, synon_dict)
    company_context = extract_company_context(words, mentioned_companies, window_size=4)
    if (row.issuerid in company_context):
      tokens.append(company_context[row.issuerid])
      labels.append(row.SentimentScore)


labels

[2,
 4,
 4,
 5,
 2,
 2,
 2,
 2,
 3,
 2,
 3,
 3,
 3,
 4,
 3,
 4,
 3,
 2,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 5,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 3,
 5,
 5,
 5,
 3,
 5,
 5,
 3,
 5,
 3,
 3,
 3,
 4,
 3,
 4,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 0,
 4,
 4,
 4,
 5,
 4,
 4,
 2,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 2,
 4,
 4,
 3,
 2,
 3,
 3,
 1,
 3,
 3,
 5,
 1,
 5,
 5,
 1,
 3,
 5,
 1,
 3,
 1,
 3,
 4,
 4,
 3,
 4,
 4,
 4,
 3,
 5,
 5,
 3,
 1,
 5,
 3,
 3,
 3,
 2,
 5,
 5,
 3,
 5,
 2,
 2,
 3,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 2,
 4,
 5,
 4,
 2,
 4,
 5,
 4,
 4,
 4,
 4,
 5,
 4,
 4,
 4,
 2,
 4,
 4,
 3,
 3,
 4,
 4,
 3,
 2,
 4,
 2,
 4,
 3,
 3,
 3,
 3,
 2,
 3,
 2,
 4,
 3,
 2,
 4,
 4,
 3,
 3,
 2,
 2,


In [5]:
from keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Embedding, LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.layers import Lambda
import keras.backend as K

max_features = 100000
maxlen = 100
batch_size = 64

In [6]:
# Инициализируем токенизатор
tokenizer = Tokenizer(num_words=max_features)

# Обучаем токенизатор на твитах
tokenizer.fit_on_texts(tokens)

# Преобразуем твиты в последовательности чисел
sequences = tokenizer.texts_to_sequences(tokens)

# Паддинг последовательностей до одинаковой длины
X = sequence.pad_sequences(sequences, maxlen=maxlen)

# Преобразуем labels в категориальный формат
y = to_categorical(labels)


# Убедитесь, что форма 'y' соответствует ожидаемой моделью
print("Форма y после to_categorical:", y.shape)

Форма y после to_categorical: (8549, 6)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
from keras import backend as K

model = Sequential()
model.add(Embedding(max_features, 256, input_length=maxlen))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=15,
    verbose=1
)

result = model.predict(X)
result

Epoch 1/15
107/107 [==============================] - 15s 91ms/step - loss: 1.2931 - accuracy: 0.4533
Epoch 2/15
107/107 [==============================] - 6s 59ms/step - loss: 0.9514 - accuracy: 0.6442
Epoch 3/15
107/107 [==============================] - 5s 43ms/step - loss: 0.6561 - accuracy: 0.7776
Epoch 4/15
107/107 [==============================] - 4s 34ms/step - loss: 0.4596 - accuracy: 0.8468
Epoch 5/15
107/107 [==============================] - 4s 36ms/step - loss: 0.3676 - accuracy: 0.8791
Epoch 6/15
107/107 [==============================] - 2s 22ms/step - loss: 0.2963 - accuracy: 0.9066
Epoch 7/15
107/107 [==============================] - 2s 21ms/step - loss: 0.2451 - accuracy: 0.9228
Epoch 8/15
107/107 [==============================] - 2s 17ms/step - loss: 0.2032 - accuracy: 0.9362
Epoch 9/15
107/107 [==============================] - 3s 24ms/step - loss: 0.1759 - accuracy: 0.9412
Epoch 10/15
107/107 [==============================] - 2s 22ms/step - loss: 0.1644 - accur

array([[4.29989130e-04, 7.65684090e-05, 1.47326097e-01, 8.51120055e-01,
        9.68767330e-04, 7.85481971e-05],
       [2.17465094e-05, 2.62777553e-06, 1.06796986e-04, 2.41685752e-03,
        9.97333288e-01, 1.18631353e-04],
       [2.96234248e-06, 7.14271778e-07, 7.89647311e-05, 1.41358149e-04,
        9.99613941e-01, 1.62088632e-04],
       ...,
       [6.02438122e-05, 2.93497806e-06, 1.15009083e-04, 9.99629021e-01,
        1.80006668e-04, 1.27975673e-05],
       [4.10501852e-05, 4.84514158e-06, 1.23246136e-04, 4.49298183e-03,
        9.95193124e-01, 1.44755395e-04],
       [8.04443425e-06, 1.47973245e-04, 2.35356056e-05, 8.50900906e-05,
        3.09801777e-04, 9.99425650e-01]], dtype=float32)

In [ ]:
# Сохранение модели
model.save('my_model.h5')

# Загрузка модели
loaded_model = load_model('my_model.h5')

# Предположим, что у вас есть вторая модель model2, уже загруженная и готовая к использованию
# Пример использования двух моделей для ансамблирования
prediction1 = loaded_model.predict(X)
prediction2 = model2.predict(X)

# Среднее предсказание
average_prediction = (prediction1 + prediction2) / 2

print("Среднее предсказание: ", average_prediction)

In [9]:
import numpy as np

# Получение предсказаний модели
predictions = model.predict(X_test)

# Преобразование предсказаний в индексы классов
predicted_classes = np.argmax(predictions, axis=1)

# Преобразование истинных меток в индексы классов, если они в формате one-hot encoding
true_classes = np.argmax(y_test, axis=1)

# Расчет точности
accuracy_manual = np.mean(predicted_classes == true_classes)
print(f"Рассчитанная вручную точность: {accuracy_manual:.2f}")


54/54 [==============================] - 0s 5ms/step
Рассчитанная вручную точность: 0.60
